In [118]:

from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer, text
from sqlalchemy.dialects.mysql import insert as mysql_insert
import pandas as pd


In [113]:
data=pd.read_csv(r"C:\Users\Hi\Downloads\Choco_Crunch.csv")

In [120]:
DB_USER = "root"
DB_PASS = "Allah786"
DB_HOST = "localhost"           
DB_PORT = 3306
DB_NAME = "choco_crunch"

In [121]:
# SQLite database file
server_engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/", echo=False)
with server_engine.begin() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS `{DB_NAME}` CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"))

In [122]:
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}", echo=False)
metadata = MetaData()

In [123]:
# Define Tables
product_info = Table(
    "product_info", metadata,
    Column("product_code", String(50), primary_key=True),
    Column("product_name", String(255)),
    Column("brand", String(255))
)

nutrient_info = Table(
    "nutrient_info", metadata,
    Column("product_code", String(50), primary_key=True),
    Column("energy_kcal_value", Float),
    Column("energy_kj_value", Float),
    Column("carbohydrates_value", Float),
    Column("sugars_value", Float),
    Column("fat_value", Float),
    Column("saturated_fat_value", Float),
    Column("proteins_value", Float),
    Column("fiber_value", Float),
    Column("salt_value", Float),
    Column("sodium_value", Float),
    Column("fruits_vegetables_nuts_estimate_from_ingredients_100g", Float),
    Column("nutrition_score_fr", Integer),
    Column("nova_group", Integer)
)

derived_metrics = Table(
    "derived_metrics", metadata,
    Column("product_code", String(50), primary_key=True),
    Column("sugar_to_carb_ratio", Float),
    Column("calorie_category", String(50)),
    Column("sugar_category", String(50)),
    Column("is_ultra_processed", String(10))
)

# Create tables (will create if not exists)
metadata.create_all(engine)

In [125]:
def row_to_val(d, key):
    v = d.get(key)
    return None if (pd.isna(v)) else v


In [127]:
from tqdm import tqdm

In [128]:
# Insert Data into Tables
with engine.begin() as conn:
    for _, row in tqdm(data.iterrows(), total=len(data)):
        # 1) product_info upsert (insert or update product_name / brand)
        pi_stmt = mysql_insert(product_info).values(
            product_code = row_to_val(row, "code"),
            product_name = row_to_val(row, "product_name"),
            brand = row_to_val(row, "brands")
        )
        pi_upsert = pi_stmt.on_duplicate_key_update(
            product_name = pi_stmt.inserted.product_name,
            brand = pi_stmt.inserted.brand
        )
        conn.execute(pi_upsert)

        # 2) nutrient_info upsert
        ni_stmt = mysql_insert(nutrient_info).values(
            product_code = row_to_val(row, "code"),
            energy_kcal_value = row_to_val(row, "energy-kcal_value"),
            energy_kj_value = row_to_val(row, "energy-kj_value"),
            carbohydrates_value = row_to_val(row, "carbohydrates_value"),
            sugars_value = row_to_val(row, "sugars_value"),
            fat_value = row_to_val(row, "fat_value"),
            saturated_fat_value = row_to_val(row, "saturated-fat_value"),
            proteins_value = row_to_val(row, "proteins_value"),
            fiber_value = row_to_val(row, "fiber_value"),
            salt_value = row_to_val(row, "salt_value"),
            sodium_value = row_to_val(row, "sodium_value"),
            fruits_vegetables_nuts_estimate_from_ingredients_100g = row_to_val(row, "fruits-vegetables-nuts-estimate-from-ingredients_100g"),
            nutrition_score_fr = row_to_val(row, "nutrition-score-fr"),
            nova_group = row_to_val(row, "nova-group")
        )
        ni_upsert = ni_stmt.on_duplicate_key_update(
            energy_kcal_value = ni_stmt.inserted.energy_kcal_value,
            energy_kj_value = ni_stmt.inserted.energy_kj_value,
            carbohydrates_value = ni_stmt.inserted.carbohydrates_value,
            sugars_value = ni_stmt.inserted.sugars_value,
            fat_value = ni_stmt.inserted.fat_value,
            saturated_fat_value = ni_stmt.inserted.saturated_fat_value,
            proteins_value = ni_stmt.inserted.proteins_value,
            fiber_value = ni_stmt.inserted.fiber_value,
            salt_value = ni_stmt.inserted.salt_value,
            sodium_value = ni_stmt.inserted.sodium_value,
            fruits_vegetables_nuts_estimate_from_ingredients_100g = ni_stmt.inserted.fruits_vegetables_nuts_estimate_from_ingredients_100g,
            nutrition_score_fr = ni_stmt.inserted.nutrition_score_fr,
            nova_group = ni_stmt.inserted.nova_group
        )
        conn.execute(ni_upsert)

        # 3) derived_metrics upsert
        dm_stmt = mysql_insert(derived_metrics).values(
            product_code = row_to_val(row, "code"),
            sugar_to_carb_ratio = row_to_val(row, "sugar_to_carb_ratio"),
            calorie_category = row_to_val(row, "calorie_category"),
            sugar_category = row_to_val(row, "sugar_category"),
            is_ultra_processed = str(row_to_val(row, "is_ultra_processed")) if row_to_val(row, "is_ultra_processed") is not None else None
        )
        dm_upsert = dm_stmt.on_duplicate_key_update(
            sugar_to_carb_ratio = dm_stmt.inserted.sugar_to_carb_ratio,
            calorie_category = dm_stmt.inserted.calorie_category,
            sugar_category = dm_stmt.inserted.sugar_category,
            is_ultra_processed = dm_stmt.inserted.is_ultra_processed
        )
        conn.execute(dm_upsert)


100%|███████████████████████████████████████████████████████████████████████████| 11997/11997 [00:48<00:00, 246.47it/s]


In [129]:
type(product_info)

sqlalchemy.sql.schema.Table

In [130]:
derived_metrics

Table('derived_metrics', MetaData(), Column('product_code', String(length=50), table=<derived_metrics>, primary_key=True, nullable=False), Column('sugar_to_carb_ratio', Float(), table=<derived_metrics>), Column('calorie_category', String(length=50), table=<derived_metrics>), Column('sugar_category', String(length=50), table=<derived_metrics>), Column('is_ultra_processed', String(length=10), table=<derived_metrics>), schema=None)

In [131]:
Product_Table = pd.read_sql(select(product_info), engine)
Nutrient_Table = pd.read_sql(select(nutrient_info), engine)
Derived_metrics_Table = pd.read_sql(select(derived_metrics), engine)


In [134]:
Product_Table

,product_code,product_name,brand
0,10001295,meat feast pizza,co op
1,10001417,Tesco Cashew Nuts,tesco
2,100035000430,lurpak plant based,Lurpak
3,10022708,None,"Dreistern, S.Pellegrino"
4,10023594,None,Suzi Wan
...,...,...,...
11836,96619885725,Extra Fancy Salted Mixed Nuts,Kirkland Signature
11837,96619955886,Maple syrup,Kirkland Signature
11838,972376,Greek style live yogurt,Marks & Spencer
11839,9780201379624,olivan citron,"Instick,mingyan food"


In [135]:
Nutrient_Table

,product_code,energy_kcal_value,energy_kj_value,carbohydrates_value,sugars_value,fat_value,saturated_fat_value,proteins_value,fiber_value,salt_value,sodium_value,fruits_vegetables_nuts_estimate_from_ingredients_100g,nutrition_score_fr,nova_group
0,10001295,485.000,719.0,65.0000,17.00000,13.0000,2.70000,25.0000,4.60000,2.2325,0.893,0.0000,4,4
1,10001417,187.000,1455.0,5.6000,1.70000,15.3000,3.00000,6.2000,1.30000,0.0100,0.004,100.0000,-3,1
2,100035000430,32.000,2001.0,29.4695,11.39940,14.8739,1.40000,5.6000,2.50000,18.5098,0.000,-1.0000,0,1
3,10022708,57.000,240.0,8.3000,0.80000,1.3000,0.30000,2.5000,2.50000,1.1000,0.440,-1.0000,0,4
4,10023594,68.000,2049.0,5.3000,0.50000,0.5000,0.10000,9.0000,0.50000,17.1000,6.840,97.8332,15,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11836,96619885725,607.143,217.0,21.4286,3.57143,57.1429,7.14286,17.8571,7.14286,758.9290,303.571,97.3214,24,3
11837,96619955886,270.000,1390.0,67.2200,58.59000,0.1400,0.03600,0.0000,0.00000,0.0225,0.009,0.0000,18,4
11838,972376,124.000,375.0,5.4000,4.80000,9.6000,6.50000,4.1000,0.50000,0.2300,0.092,0.0000,8,1
11839,9780201379624,371.000,153.0,74.6700,2.57000,1.5300,0.28000,0.0000,0.00000,0.0300,0.012,0.0000,0,4


In [136]:
Derived_metrics_Table

,product_code,sugar_to_carb_ratio,calorie_category,sugar_category,is_ultra_processed
0,10001295,0.261538,High,High Sugar,Yes
1,10001417,0.303571,Moderate,Low Sugar,No
2,100035000430,0.386821,Low,Moderate Sugar,No
3,10022708,0.096385,Low,Low Sugar,Yes
4,10023594,0.094340,Low,Low Sugar,Yes
...,...,...,...,...,...
11836,96619885725,0.166667,High,Low Sugar,No
11837,96619955886,0.871616,Moderate,High Sugar,Yes
11838,972376,0.888889,Low,Low Sugar,No
11839,9780201379624,0.034418,Moderate,Low Sugar,Yes


In [52]:
#PRODUCT INFO QUERIES

product_Query_df = pd.read_sql(select(product_info), engine)

In [66]:
#1. Count products per brand

product_Query_df.groupby('brand')['product_name'].nunique()

brand
07 x Aldi 09.25, 09 x Aldi 09.25, 11 x Aldi 09.25, 12 x Aldi 09.25-01 x Aldi 08.25, Milsani, 05 x Aldi 09.25             1
1001 Delights (Lidl), Envia, Lidl, Milbona, Milbona (Lidl)                                                               1
1540 Vestby, Norwegen, Oluf Lorentzen AS, Toveien 22, Mutti                                                              1
166, 43122 Parma - Italy, Barilla, Barilla - Barilla G. e R. Fratelli - Società per Azioni - Via Mantova, 166 Barilla    1
1664 Blonde, Kronenbourg                                                                                                 1
                                                                                                                        ..
شرقي,Chergui                                                                                                             1
كنور                                                                                                                     1
كوسومار   

In [54]:
#2. Count unique products per brand

product_Query_df['brand'].value_counts()

brand
Hacendado                     366
Tesco                         292
Carrefour                     147
Heinz                         100
Sainsbury's                    99
                             ... 
MIXE                            1
saint alby                      1
Marque Repère,Notre Jardin      1
Côte d'Or,Mondelēz              1
SCHOLETTA                       1
Name: count, Length: 3990, dtype: int64

In [55]:
#3. Top 5 brands by product count

product_Query_df['brand'].value_counts().head(5)

brand
Hacendado      366
Tesco          292
Carrefour      147
Heinz          100
Sainsbury's     99
Name: count, dtype: int64

In [67]:
#self_query

brand_counts =product_Query_df['brand'].value_counts()

brands_with_one = brand_counts[brand_counts == 1]

In [68]:
brands_with_one 
# 2598 brands have only one count

brand
Chabrior,Sélection Intermarché                1
daunat                                        1
Eau de Beckerich, Rocheval, Source Ophélie    1
Materne, Pom'potes                            1
Nos Régions ont du Talent                     1
                                             ..
MIXE                                          1
saint alby                                    1
Marque Repère,Notre Jardin                    1
Côte d'Or,Mondelēz                            1
SCHOLETTA                                     1
Name: count, Length: 2598, dtype: int64

In [26]:
# 4. Products with missing product name

product_Query_df[product_Query_df['product_name'].isnull()]

,product_code,product_name,brand
14,6111242100985,None,Jaouda
16,6111099003897,None,lilia
38,6111128000460,None,aïn Saiss
43,20005733,None,"Alesto,Lidl"
58,8425197712024,None,Maruja
...,...,...,...
11626,40144016,None,WERTHER'S ORIGINAL
11665,5900617002266,None,Sante
11666,8410376052633,None,Gullón
11768,168069,None,Marks & Spencer


In [ ]:
data[["product_name", "brands"]] = data[["product_name", "brands"]].ffill()

In [59]:
product_Query_df[product_Query_df['product_name'].isnull()]

,product_code,product_name,brand


In [69]:
# 5. Number of unique brands

product_Query_df['brand'].nunique()

3990

In [72]:
# 6. Products with code starting with '3'

product_Query_df[product_Query_df['product_code'].str.startswith('3')]

,product_code,product_name,brand
6,3274080005003,Eau De Source,Cristaline
20,3046920029759,Lindt Excellence 90%,Lindt&Sprüngli
21,3046920028004,Excellence 70% Cocoa Intense Dark,Lindt
23,3017620425035,Nutella,Ferrero
28,3175680011480,Sésame,Gerblé
...,...,...,...
11801,3362600003100,Cigares fourrés Cacao,Primal Kitchen
11815,3185200001327,Croutons de pains croustillants,Tipiak
11816,3450601046254,3450601046254,l'arbre vert
11825,3088541015209,Salted Sweet Cream Butter,Lune de Miel


In [73]:
### Nutrient_info_QUERIES

nutrient_query_df = pd.read_sql(select(nutrient_info), engine)

In [74]:
# 7. Top 10 products with highest energy-kcal_value

nutrient_query_df.nlargest(10, 'energy_kcal_value')

,product_code,energy_kcal_value,energy_kj_value,carbohydrates_value,sugars_value,fat_value,saturated_fat_value,proteins_value,fiber_value,salt_value,sodium_value,fruits_vegetables_nuts_estimate_from_ingredients_100g,nutrition_score_fr,nova_group
5348,6111261970903,45793.499044,1916.0,29.469532,11.399435,14.873864,1.4,5.6,2.5,18.509833,0.000,-1.000,0,4
5349,8480000054647,45793.499044,1916.0,3260.000000,40.000000,3400.000000,400.0,3000.0,3000.0,0.000000,0.000,0.000,17,1
2688,6111259090132,5280.000000,1.0,63.000000,0.000000,40.000000,52.0,8.0,0.0,20.000000,8.000,0.000,27,4
221,3088543506255,1600.000000,802.0,402.500000,395.000000,2.500000,0.0,2.5,2.5,0.000000,0.000,0.000,20,1
11916,72486002205,990.000000,1945.0,168.000000,45.000000,30.000000,12.0,12.0,6.0,5.175000,2.070,0.000,7,4
10604,36000291452,904.000000,2326.0,55.000000,3.000000,42.000000,19.0,72.0,3.0,2.067500,0.827,60.000,0,4
112,6111099000599,900.000000,3768.0,0.000000,0.000000,100.000000,14.0,0.0,0.0,18.509833,0.048,-1.000,0,2
192,6111024001516,900.000000,2285.0,0.000000,0.000000,100.000000,15.0,0.0,2.5,0.000000,0.000,14.375,5,4
392,6111024002186,900.000000,3700.0,0.000000,0.000000,100.000000,12.6,0.0,0.0,0.000000,0.000,0.000,-1,4
493,6111099000582,900.000000,1791.0,0.000000,0.000000,100.000000,14.0,0.0,2.5,0.000000,0.000,-1.000,0,2


In [75]:
# 8. Average sugars_value per nova-group

nutrient_query_df.groupby('nova_group')['sugars_value'].mean()

nova_group
1     6.741651
2    17.365458
3     7.190336
4    13.264616
Name: sugars_value, dtype: float64

In [76]:
# 9. Count products with fat_value > 20g

(nutrient_query_df['fat_value'] > 20).sum()

np.int64(2912)

In [77]:
# 10. Average carbohydrates_value per product

nutrient_query_df['carbohydrates_value'].mean()

np.float64(29.469532177154438)

In [78]:
# 11. Products with sodium_value > 1g

nutrient_query_df[nutrient_query_df['sodium_value'] > 1]

,product_code,energy_kcal_value,energy_kj_value,carbohydrates_value,sugars_value,fat_value,saturated_fat_value,proteins_value,fiber_value,salt_value,sodium_value,fruits_vegetables_nuts_estimate_from_ingredients_100g,nutrition_score_fr,nova_group
1,6111242100992,97.0,406.00,9.400000,11.399435,3.000000,1.4,8.0,2.5,18.509833,26.00,0.000000,0,3
2,6111035002175,0.0,0.00,29.469532,11.399435,14.873864,1.4,5.6,2.5,65.000000,26.00,0.000000,0,4
3,6111035000058,0.0,0.00,29.469532,11.399435,14.873864,1.4,5.6,2.5,65.000000,26.00,0.000000,0,1
6,3274080005003,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,52.500000,21.00,0.000000,0,1
13,50184453,260.0,1100.00,30.000000,1.200000,0.500000,0.1,34.0,2.5,10.800000,4.32,0.734863,23,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11927,6194001800302,47.0,198.00,29.469532,11.399435,14.873864,1.4,5.6,2.5,91.750000,36.70,0.000000,0,1
11956,863699000108,100.0,1016.65,0.000000,0.000000,11.000000,1.5,0.0,0.0,212.500000,85.00,0.000000,25,3
11980,767707001678,100.0,791.00,0.000000,0.000000,11.000000,7.0,0.0,0.0,250.000000,100.00,0.000000,27,2
11981,3088541015209,320.0,791.00,80.000000,62.000000,0.000000,1.4,0.0,2.5,18.509833,4.60,0.000000,0,2


In [79]:
# 12. Count products with non-zero fruits-vegetables-nuts content

(nutrient_query_df['fruits_vegetables_nuts_estimate_from_ingredients_100g'] > 0).sum()

np.int64(5789)

In [80]:
# 13. Products with energy-kcal_value > 500

nutrient_query_df[nutrient_query_df['energy_kcal_value'] > 500]

,product_code,energy_kcal_value,energy_kj_value,carbohydrates_value,sugars_value,fat_value,saturated_fat_value,proteins_value,fiber_value,salt_value,sodium_value,fruits_vegetables_nuts_estimate_from_ingredients_100g,nutrition_score_fr,nova_group
16,6111099003897,675.0,180.0,0.200000,0.000000,75.000000,34.0,0.1,0.0,0.50000,0.2000,0.0,18,4
21,3046920028004,566.0,2350.0,34.000000,30.000000,41.000000,24.0,9.5,2.5,0.10000,0.0400,0.0,25,3
22,6111035003035,539.0,2252.0,29.469532,11.399435,14.873864,1.4,5.6,2.5,0.78375,0.3135,0.0,0,4
23,3017620425035,539.0,2252.0,57.500000,56.300000,30.900000,10.6,6.3,2.5,0.10700,0.0428,13.0,31,4
33,3046920028363,584.0,2418.0,22.000000,14.000000,46.000000,27.0,12.5,2.5,0.02000,0.0080,0.0,20,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11940,5060040253731,539.0,2250.0,61.000000,0.600000,31.000000,1.8,2.4,2.9,0.55000,0.2200,24.5,9,4
11966,6111203005038,514.0,167.0,0.500000,0.000000,57.000000,0.0,0.5,2.5,0.00000,0.0000,-1.0,0,2
11976,5000119007247,538.0,2245.0,52.500000,0.500000,33.200000,3.0,5.6,3.6,1.00000,0.4000,12.5,11,3
11979,4066447423761,824.0,3389.0,0.000000,0.000000,92.000000,13.0,0.0,0.0,0.00000,0.0000,100.0,0,2


In [81]:
### DERIVED METRICS QUERIES

derived_query_df = pd.read_sql(select(derived_metrics), engine)


In [82]:
# 14. Count products per calorie_category

derived_query_df['calorie_category'].value_counts()

calorie_category
Low         4474
Moderate    4050
High        3473
Name: count, dtype: int64

In [83]:
# 15. Count of High Sugar products

(derived_query_df['sugar_category'] == 'High Sugar').sum()

np.int64(2566)

In [85]:
# 16. Average sugar_to_carb_ratio for High Calorie products

derived_query_df[derived_query_df['calorie_category']=='High Calorie']['sugar_to_carb_ratio'].mean()

nan

In [86]:
# 17. Products that are both High Calorie and High Sugar

derived_query_df[(derived_query_df['calorie_category']=='High Calorie') & 
           (derived_query_df['sugar_category']=='High Sugar')]

,product_code,sugar_to_carb_ratio,calorie_category,sugar_category,is_ultra_processed


In [87]:
# 18. Number of products marked as ultra-processed

(derived_query_df['is_ultra_processed']==True).sum()

np.int64(0)

In [92]:
# 19. Products with sugar_to_carb_ratio > 0.7

derived_query_df[derived_query_df['sugar_to_carb_ratio'] > 0.7]

,product_code,sugar_to_carb_ratio,calorie_category,sugar_category,is_ultra_processed
1,6111242100992,1.212706,Low,Moderate Sugar,NaN
17,5449000054227,1.000000,Low,Moderate Sugar,NaN
21,3046920028004,0.882353,High,High Sugar,NaN
23,3017620425035,0.979130,High,High Sugar,NaN
24,5449000147417,1.827957,Low,High Sugar,NaN
...,...,...,...,...,...
11988,6111242923850,0.990909,Low,Moderate Sugar,NaN
11990,7622201768836,0.910714,High,High Sugar,NaN
11991,4056489774747,0.939498,Moderate,High Sugar,NaN
11994,5010478014510,0.750000,Moderate,High Sugar,NaN


In [93]:
# 20. Average sugar_to_carb_ratio per calorie_category

derived_query_df.groupby('calorie_category')['sugar_to_carb_ratio'].mean()

calorie_category
High         0.466257
Low         32.512468
Moderate     0.482388
Name: sugar_to_carb_ratio, dtype: float64

In [95]:
### JOINED QUERIES

merged = product_Query_df.merge(derived_query_df, on='product_code', how='left') \
                   .merge(nutrient_query_df, on='product_code', how='left')

In [96]:
# 21. Top 5 brands with most High Calorie products

merged[merged['calorie_category']=='High Calorie']['brand'].value_counts().head(5)

Series([], Name: count, dtype: int64)

In [97]:
# 22. Average energy_kcal_value for each calorie_category

merged.groupby('calorie_category')['energy_kcal_value'].mean()

calorie_category
High        562.058277
Low          68.916842
Moderate    277.573576
Name: energy_kcal_value, dtype: float64

In [98]:
# 23. Count of ultra-processed products per brand

merged[merged['is_ultra_processed']==True].groupby('brand').size()

Series([], dtype: int64)

In [99]:
# 24. Products with High Sugar and High Calorie along with brand

merged[(merged['calorie_category']=='High Calorie') & 
       (merged['sugar_category']=='High Sugar')][['product_name','brand']]

,product_name,brand


In [100]:
# 25. Average sugar content per brand for ultra-processed products

merged.groupby('calorie_category')['energy_kcal_value'].mean()

calorie_category
High        562.058277
Low          68.916842
Moderate    277.573576
Name: energy_kcal_value, dtype: float64

In [101]:
# 26. Number of products with fruits/vegetables/nuts content in each calorie_category

merged[merged['fruits_vegetables_nuts_estimate_from_ingredients_100g']>0].groupby('calorie_category').size()

calorie_category
High        1791
Low         2318
Moderate    1826
dtype: int64

In [102]:
# 27. Top 5 products by sugar_to_carb_ratio with their calorie and sugar category

merged.nlargest(5, 'sugar_to_carb_ratio')[['product_name','calorie_category','sugar_category','sugar_to_carb_ratio']]

,product_name,calorie_category,sugar_category,sugar_to_carb_ratio
3811,Frylight Sunflower Oil Spray 190ml,Low,Moderate Sugar,142492.934665
12143,Emmental Milbona,Moderate,Moderate Sugar,113.994348
2187,Assaisonnement au citron,Low,Moderate Sugar,81.424534
9699,"Gelatina 0% cereza, cola y sandía",Low,Moderate Sugar,56.997174
6570,Mozzarella,Low,Moderate Sugar,54.283023
